# COGS 108 - EDA Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Setup

In [73]:
# import packages and setup visuals
import pandas as pd
import numpy as np
import os

In [74]:
# merge all contract files in contracts directory as one dataframe
directory = 'data/contracts/'
filepaths = [directory + filename for filename in os.listdir(directory)]
contracts = pd.concat([pd.read_csv(filepath) for filepath in filepaths])
print('contracts shape: ', contracts.shape)

contracts shape:  (4996, 18)


# Data Cleaning

Describe your data cleaning steps here.

In [75]:
display(contracts.head())
display(batting.head())

,Player,Pos'n,Age,Qual Offer,Old Club,New Club,Years,Guarantee,Term,Option,Opt Out,AAV,Player Agent,Club Owner,Baseball Ops head / club GM,Details,Age 7/1/21,Age 7/1/22
0,"Cano, Robinson",2b,31.0,rejected,NYA,SEA,10.0,"$240,000,000",2014-23,NaN,NaN,"$24,000,000",Roc Nation,Nintendo/America,Jack Zduriencik,NaN,NaN,NaN
1,"Ellsbury, Jacoby",of,30.0,rejected,BOS,NYA,7.0,"$153,000,000",2014-20,club,NaN,"$21,857,143",Boras Corp.,Hal Steinbrenner,Brian Cashman,NaN,NaN,NaN
2,"Choo, Shin-Soo",rf,31.0,rejected,CIN,TEX,7.0,"$130,000,000",2014-20,NaN,NaN,"$18,571,429",Boras Corp.,RDavis/BSimpson,Jon Daniels,NaN,NaN,NaN
3,"McCann, Brian",c,30.0,rejected,ATL,NYA,5.0,"$85,000,000",2014-18,club,NaN,"$17,000,000",BB Abbott,Hal Steinbrenner,Brian Cashman,NaN,NaN,NaN
4,"Granderson, Curtis",of,33.0,rejected,NYA,NYN,4.0,"$60,000,000",2014-17,NaN,NaN,"$15,000,000",Matt Brown,Fred Wilpon,Sandy Alderson,NaN,NaN,NaN


,playerID,year,AB,AVG,OBS,SLG,OBS,playerName
0,aardsda01,2006,2,0.000,0.000,0.000,0.000,David Aardsma
1,aardsda01,2008,1,0.000,0.000,0.000,0.000,David Aardsma
2,aardsda01,2015,1,0.000,0.000,0.000,0.000,David Aardsma
3,aaronha01,1954,468,0.280,0.769,0.447,0.769,Hank Aaron
4,aaronha01,1955,602,0.314,0.906,0.540,0.906,Hank Aaron


In [76]:
# drop unecessary columns
contracts = contracts[['Player','Pos\'n', 'Age', 'Term', 'AAV']]

# drop players that are pitchers or have NaN values
contracts = contracts[contracts["Pos'n"].str.contains("hp") == False]
contracts = contracts.dropna(axis=0)

# drop position column (no longer needed)
contracts = contracts.drop('Pos\'n',axis=1)
display(contracts)

,Player,Age,Term,AAV
0,"Cano, Robinson",31.0,2014-23,"$24,000,000"
1,"Ellsbury, Jacoby",30.0,2014-20,"$21,857,143"
2,"Choo, Shin-Soo",31.0,2014-20,"$18,571,429"
3,"McCann, Brian",30.0,2014-18,"$17,000,000"
4,"Granderson, Curtis",33.0,2014-17,"$15,000,000"
...,...,...,...,...
66,"Chisenhall, Lonnie",30.0,2019,"$2,750,000"
70,"Maldonado, Martin",32.0,2019,"$2,500,000"
75,"McCann, Brian",35.0,2019,"$2,000,000"
77,"Harrison, Josh",31.0,2019,"$2,000,000"


In [77]:
# functions to standardize variables
def salary_to_int(str_in):
    return int(str_in.replace('$','').replace(',',''))

def term_to_year(str_in):
    return int(str(str_in).split('-')[0])

In [78]:
# standardize salary and term variables
contracts['AAV'] = contracts['AAV'].apply(salary_to_int)
contracts['Term'] = contracts['Term'].apply(term_to_year)
contracts

,Player,Age,Term,AAV
0,"Cano, Robinson",31.0,2014,24000000
1,"Ellsbury, Jacoby",30.0,2014,21857143
2,"Choo, Shin-Soo",31.0,2014,18571429
3,"McCann, Brian",30.0,2014,17000000
4,"Granderson, Curtis",33.0,2014,15000000
...,...,...,...,...
66,"Chisenhall, Lonnie",30.0,2019,2750000
70,"Maldonado, Martin",32.0,2019,2500000
75,"McCann, Brian",35.0,2019,2000000
77,"Harrison, Josh",31.0,2019,2000000


In [79]:
# rename columns for consistency
contracts.columns = ['playerName','playerAge','year','yearSalary']
contracts.head()

,playerName,playerAge,year,yearSalary
0,"Cano, Robinson",31.0,2014,24000000
1,"Ellsbury, Jacoby",30.0,2014,21857143
2,"Choo, Shin-Soo",31.0,2014,18571429
3,"McCann, Brian",30.0,2014,17000000
4,"Granderson, Curtis",33.0,2014,15000000


In [80]:
# read people and batting files as dataframes
people = pd.read_csv('data/batting/People.csv')
batting = pd.read_csv('data/batting/Batting.csv')
print('people shape: ', people.shape)
print('batting shape: ', batting.shape)

people shape:  (20370, 24)
batting shape:  (110495, 22)


In [81]:
batting = batting[['playerID', 'yearID', 'AB', 'H', '2B', '3B', 'HR', 'BB','HBP','SF']]

In [82]:
display(batting[batting['playerID'] == 'abramca01'])
batting = batting.groupby(['playerID', 'yearID']).agg(lambda x: x.sum(min_count=1))
batting = batting.reset_index()
display(batting[batting['playerID'] == 'abramca01'])

,playerID,yearID,AB,H,2B,3B,HR,BB,HBP,SF
33904,abramca01,1949,24,2,1,0,0,7,0.0,NaN
34477,abramca01,1950,44,9,1,0,0,9,0.0,NaN
35053,abramca01,1951,150,42,8,0,3,36,0.0,NaN
35670,abramca01,1952,10,2,0,0,0,2,0.0,NaN
35671,abramca01,1952,158,44,9,2,2,19,0.0,NaN
36303,abramca01,1953,448,128,10,6,15,58,0.0,NaN
36889,abramca01,1954,42,6,1,1,0,10,0.0,0.0
36890,abramca01,1954,423,124,22,7,6,72,4.0,1.0
37466,abramca01,1955,309,75,12,3,6,89,3.0,3.0
38121,abramca01,1956,3,1,0,0,0,2,0.0,0.0


,playerID,yearID,AB,H,2B,3B,HR,BB,HBP,SF
187,abramca01,1949,24,2,1,0,0,7,0.0,NaN
188,abramca01,1950,44,9,1,0,0,9,0.0,NaN
189,abramca01,1951,150,42,8,0,3,36,0.0,NaN
190,abramca01,1952,168,46,9,2,2,21,0.0,NaN
191,abramca01,1953,448,128,10,6,15,58,0.0,NaN
192,abramca01,1954,465,130,23,8,6,82,4.0,1.0
193,abramca01,1955,309,75,12,3,6,89,3.0,3.0
194,abramca01,1956,3,1,0,0,0,2,0.0,0.0


In [83]:
# drop observations with NaN values
batting = batting.dropna(axis=0)

# drop observations with 0 At Bats
batting = batting[batting['AB'] > 0]
batting.shape

(53054, 10)

In [84]:
# functions used for calculating batting average and OBS average
def calc_avg(h, ab):
    return h / ab
    
def calc_obp(h, bb, hbp, ab, sf):
    return (h + bb + hbp) / (ab + bb + sf + hbp)
    
def calc_tb(h, two_b, three_b, hr):
    singles = h - two_b - three_b - hr
    return singles + two_b * 2 + three_b * 3 + hr * 4

def calc_slg(tb, ab):
    return tb / ab

def calc_obs(obp, slg):
    return obp + slg

In [85]:
# create Batting Average column in batting dataframe
avg = calc_avg(h=batting['H'], ab=batting['AB'])
batting['AVG'] = round(avg, 3)

# create On Base Percentage average column in batting dataframe
obp = calc_obp(h=batting['H'], bb=batting['BB'], hbp=batting['HBP'], ab=batting['AB'], sf=batting['SF'])
batting['OBP'] = round(obp, 3)

# create Slugging average column in batting dataframe
tb = calc_tb(h=batting['H'], two_b=batting['2B'], three_b=batting['3B'], hr=batting['HR'])
slg = calc_slg(tb, batting['AB'])
batting['SLG'] = round(slg, 3)

# create On Base Plus Slugging column in batting dataframe
obs = calc_obs(obp, slg)
batting['OBS'] = round(obs, 3)

batting.head()

,playerID,yearID,AB,H,2B,3B,HR,BB,HBP,SF,AVG,OBP,SLG,OBS
1,aardsda01,2006,2,0,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000
3,aardsda01,2008,1,0,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000
8,aardsda01,2015,1,0,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000
9,aaronha01,1954,468,131,27,6,13,28,3.0,4.0,0.280,0.322,0.447,0.769
10,aaronha01,1955,602,189,37,9,27,49,3.0,4.0,0.314,0.366,0.540,0.906


In [86]:
# drop irrelevant columns and rename
batting = batting[['playerID', 'yearID','AB', 'AVG', 'OBP','SLG','OBS']]
batting.columns = ['playerID', 'year', 'AB', 'AVG', 'OBP', 'SLG','OBS']

In [87]:
batting

,playerID,year,AB,AVG,OBP,SLG,OBS
1,aardsda01,2006,2,0.000,0.000,0.000,0.000
3,aardsda01,2008,1,0.000,0.000,0.000,0.000
8,aardsda01,2015,1,0.000,0.000,0.000,0.000
9,aaronha01,1954,468,0.280,0.322,0.447,0.769
10,aaronha01,1955,602,0.314,0.366,0.540,0.906
...,...,...,...,...,...,...,...
102185,zuverge01,1954,66,0.136,0.149,0.152,0.301
102186,zuverge01,1955,27,0.185,0.214,0.222,0.437
102187,zuverge01,1956,17,0.118,0.167,0.118,0.284
102188,zuverge01,1957,23,0.130,0.231,0.130,0.361


In [88]:
# add name column to people that combines players first and last name
people['name'] = people['nameFirst'] + ' ' + people['nameLast']

# drop irrelevant columns
people = people[['playerID','name']]

# merge batting and people to add name column to batting dataframe
batting = batting.merge(people, how='left', on='playerID')

# rename and reorder  batting columns for consistency 
batting = batting.rename({'name':'playerName'}, axis=1)
batting = batting[['playerID','playerName','year','AB', 'AVG', 'OBP', 'SLG','OBS']]
batting.head()

,playerID,playerName,year,AB,AVG,OBP,SLG,OBS
0,aardsda01,David Aardsma,2006,2,0.000,0.000,0.000,0.000
1,aardsda01,David Aardsma,2008,1,0.000,0.000,0.000,0.000
2,aardsda01,David Aardsma,2015,1,0.000,0.000,0.000,0.000
3,aaronha01,Hank Aaron,1954,468,0.280,0.322,0.447,0.769
4,aaronha01,Hank Aaron,1955,602,0.314,0.366,0.540,0.906


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [22]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION